In [1]:
import sys
import os
import wandb
import pandas as pd
!wandb off
# wandb.login()

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


W&B offline, running your script from this directory will only write metadata locally.


In [2]:
!mamba create -n py312 -y
!source /opt/conda/bin/activate py312 && mamba install python=3.12.2 jupyter mamba -y

!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3
!sudo rm /opt/conda/bin/python3.7
!sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3.7
!sudo rm /opt/conda/bin/python
!sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)



                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.25.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████




Looking for: []

Preparing transaction: - 

done
Verifying transaction: | 

/ 

...

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
!git clone https://github.com/Ben-Gayther/predicting_goodreads_review_ratings.git

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'predicting_goodreads_review_ratings'...


remote: Enumerating objects: 290, done.
remote: Counting objects:   0% (1/290)
remote: Counting objects:   1% (3/290)
remote: Counting objects:   2% (6/290)
remote: Counting objects:   3% (9/290)
remote: Counting objects:   4% (12/290)
remote: Counting objects:   5% (15/290)
remote: Counting objects:   6% (18/290)
remote: Counting objects:   7% (21/290)
remote: Counting objects:   8% (24/290)
remote: Counting objects:   9% (27/290)
remote: Counting objects:  10% (29/290)
remote: Counting objects:  11% (32/290)
remote: Counting objects:  12% (35/290)
remote: Counting objects:  13% (38/290)
remote: Counting objects:  14% (41/290)
remote: Counting objects:  15% (44/290)
remote: Counting objects:  16% (47/290)
remote: Counting objects:  17% (50/290)
remote: Counting objects:  18% (53/290)
remote: Counting objects:  19% (56/290)
remote: Counting objects:  20% (58/290)
remote: Counting objects:  21% (61/290)
remote: Counting objects:  22% (64/290)
remote: Counting objects:  23% (67/290)
remo

remote: Compressing objects:   3% (6/186)
remote: Compressing objects:   4% (8/186)
remote: Compressing objects:   5% (10/186)
remote: Compressing objects:   6% (12/186)
remote: Compressing objects:   7% (14/186)
remote: Compressing objects:   8% (15/186)
remote: Compressing objects:   9% (17/186)
remote: Compressing objects:  10% (19/186)
remote: Compressing objects:  11% (21/186)
remote: Compressing objects:  12% (23/186)
remote: Compressing objects:  13% (25/186)
remote: Compressing objects:  14% (27/186)
remote: Compressing objects:  15% (28/186)
remote: Compressing objects:  16% (30/186)
remote: Compressing objects:  17% (32/186)
remote: Compressing objects:  18% (34/186)
remote: Compressing objects:  19% (36/186)
remote: Compressing objects:  20% (38/186)
remote: Compressing objects:  21% (40/186)
remote: Compressing objects:  22% (41/186)
remote: Compressing objects:  23% (43/186)
remote: Compressing objects:  24% (45/186)
remote: Compressing objects:  25% (47/186)
remote: Compr

Receiving objects:   0% (1/290)
Receiving objects:   1% (3/290)
Receiving objects:   2% (6/290)
Receiving objects:   3% (9/290)
Receiving objects:   4% (12/290)
Receiving objects:   5% (15/290)
Receiving objects:   6% (18/290)
Receiving objects:   7% (21/290)
Receiving objects:   8% (24/290)
Receiving objects:   9% (27/290)
Receiving objects:  10% (29/290)
Receiving objects:  11% (32/290)
Receiving objects:  12% (35/290)
Receiving objects:  13% (38/290)
Receiving objects:  14% (41/290)
Receiving objects:  15% (44/290)
Receiving objects:  16% (47/290)
Receiving objects:  17% (50/290)
Receiving objects:  18% (53/290)
Receiving objects:  19% (56/290)
Receiving objects:  20% (58/290)
Receiving objects:  21% (61/290)
Receiving objects:  22% (64/290)
Receiving objects:  23% (67/290)
Receiving objects:  24% (70/290)
Receiving objects:  25% (73/290)
Receiving objects:  26% (76/290)
Receiving objects:  27% (79/290)
Receiving objects:  28% (82/290)


Receiving objects:  31% (90/290)
Receiving objects:  32% (93/290)
Receiving objects:  33% (96/290)
Receiving objects:  34% (99/290)
Receiving objects:  35% (102/290)
Receiving objects:  36% (105/290)
Receiving objects:  37% (108/290)
Receiving objects:  38% (111/290)
Receiving objects:  39% (114/290)
Receiving objects:  40% (116/290)
Receiving objects:  41% (119/290)
Receiving objects:  42% (122/290)
Receiving objects:  43% (125/290)
Receiving objects:  44% (128/290)
Receiving objects:  45% (131/290)
Receiving objects:  46% (134/290)
Receiving objects:  47% (137/290)
Receiving objects:  48% (140/290)
Receiving objects:  49% (143/290)
Receiving objects:  50% (145/290)
Receiving objects:  51% (148/290)
Receiving objects:  52% (151/290)
Receiving objects:  53% (154/290)
Receiving objects:  54% (157/290)
Receiving objects:  55% (160/290)
Receiving objects:  56% (163/290)
Receiving objects:  57% (166/290)
Receiving objects:  58% (169/290)


Receiving objects:  60% (174/290)
Receiving objects:  61% (177/290)
Receiving objects:  62% (180/290)
Receiving objects:  63% (183/290)
Receiving objects:  64% (186/290)
Receiving objects:  65% (189/290)
Receiving objects:  66% (192/290)
Receiving objects:  67% (195/290)
Receiving objects:  68% (198/290)
Receiving objects:  69% (201/290)
Receiving objects:  70% (203/290)
Receiving objects:  71% (206/290)
Receiving objects:  72% (209/290)
Receiving objects:  73% (212/290)
Receiving objects:  74% (215/290)
Receiving objects:  75% (218/290)
Receiving objects:  76% (221/290)
Receiving objects:  77% (224/290)
Receiving objects:  78% (227/290)
Receiving objects:  79% (230/290)
Receiving objects:  80% (232/290)
Receiving objects:  81% (235/290)
Receiving objects:  82% (238/290)
Receiving objects:  83% (241/290)
Receiving objects:  84% (244/290)
Receiving objects:  85% (247/290)
Receiving objects:  86% (250/290)
Receiving objects:  87% (253/290)


remote: Total 290 (delta 156), reused 217 (delta 96), pack-reused 0
Receiving objects:  89% (259/290)
Receiving objects:  90% (261/290)
Receiving objects:  91% (264/290)
Receiving objects:  92% (267/290)
Receiving objects:  93% (270/290)
Receiving objects:  94% (273/290)
Receiving objects:  95% (276/290)
Receiving objects:  96% (279/290)
Receiving objects:  97% (282/290)
Receiving objects:  98% (285/290)
Receiving objects:  99% (288/290)
Receiving objects: 100% (290/290)
Receiving objects: 100% (290/290), 2.45 MiB | 7.18 MiB/s, done.
Resolving deltas:   0% (0/156)
Resolving deltas:   1% (2/156)
Resolving deltas:   5% (9/156)
Resolving deltas:   8% (13/156)
Resolving deltas:  12% (19/156)
Resolving deltas:  14% (22/156)
Resolving deltas:  15% (24/156)
Resolving deltas:  16% (26/156)
Resolving deltas:  17% (28/156)
Resolving deltas:  18% (29/156)
Resolving deltas:  19% (31/156)
Resolving deltas:  29% (46/156)
Resolving deltas:  43% (68/156)
Resolving deltas:  44% (69/156)
Resolving delta

In [4]:
os.chdir("predicting_goodreads_review_ratings")

In [5]:
yaml_contents = """
---
LOGGING:
  LEVEL: "INFO"
  FORMAT: "%(asctime)s - %(name)s - %(levelname)s - %(message)s"

TRAINING_ARGS:
  MODEL_NAME: "nlptown/bert-base-multilingual-uncased-sentiment"
  LEARNING_RATE: 2.0e-5
  MAX_LENGTH: 256
  BATCH_SIZE: 80
  EPOCHS: 1
  FULL_DATASET: True
  TEST_RUN: False
  OUTPUT_DIR: "models/"
  SUBMISSION_NAME: "/kaggle/working/submission.csv"

DATA_ARGS:
  INPUT_TRAIN_DATA: "/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv"
  OUTPUT_TRAIN_NAME: "data/processed_goodreads_train.csv"
  INPUT_TEST_DATA: "/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv"
  OUTPUT_TEST_NAME: "data/processed_goodreads_test.csv"
"""

In [6]:
with open("config.yaml", "w") as file:
    file.write(yaml_contents)

In [7]:
!pip install -r requirements.txt

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Ignoring colorama: markers 'python_version >= "3.12" and python_version < "4.0" and platform_system == "Windows"' don't match your environment


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/159.4 kB ? eta -:--:--


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 6.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/61.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00


...

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
!python src/prepare_data.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


2024-04-05 12:36:58,834 - root - INFO - Read data from /kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv, fields are Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')


2024-04-05 12:37:22,740 - root - INFO - Preprocessed data


2024-04-05 12:38:50,553 - root - INFO - Saved data to data/processed_goodreads_train.csv


2024-04-05 12:39:02,634 - root - INFO - Read data from /kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv, fields are Index(['user_id', 'book_id', 'review_id', 'review_text', 'date_added',
       'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments'],
      dtype='object')


2024-04-05 12:39:15,304 - root - INFO - Preprocessed data


2024-04-05 12:40:01,975 - root - INFO - Saved data to data/processed_goodreads_test.csv


2024-04-05 12:40:02,101 - root - INFO - Finished preprocessing data


In [9]:
!python src/train_model.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


2024-04-05 12:40:30,269 - root - INFO - Read data from data/processed_goodreads_train.csv



tokenizer_config.json:   0%|                         | 0.00/39.0 [00:00<?, ?B/s]
tokenizer_config.json: 100%|██████████████████| 39.0/39.0 [00:00<00:00, 201kB/s]



config.json:   0%|                                    | 0.00/953 [00:00<?, ?B/s]
config.json: 100%|█████████████████████████████| 953/953 [00:00<00:00, 5.40MB/s]



vocab.txt:   0%|                                     | 0.00/872k [00:00<?, ?B/s]

...


100%|██████████████████████████████████▉| 11248/11250 [5:56:39<00:03,  1.90s/it]


100%|██████████████████████████████████▉| 11249/11250 [5:56:40<00:01,  1.90s/it]


100%|███████████████████████████████████| 11250/11250 [5:56:42<00:00,  1.91s/it]


                                                                                
{'train_runtime': 21406.3737, 'train_samples_per_second': 42.044, 'train_steps_per_second': 0.526, 'train_loss': 0.9059517415364583, 'epoch': 1.0}

100%|███████████████████████████████████| 11250/11250 [5:56:46<00:00,  1.90s/it]
2024-04-05 18:45:05,241 - root - INFO - Trained model and saved results to models/nlptown/bert-base-multilingual-uncased-sentiment/train_results.csv


2024-04-05 18:45:06,557 - root - INFO - Saved transformer model and tokenizer to models/nlptown/bert-base-multilingual-uncased-sentiment


In [10]:
!python src/make_preds.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


2024-04-05 18:45:20,612 - root - INFO - Loaded model


2024-04-05 18:45:20,965 - root - INFO - Loaded tokenizer


2024-04-05 18:45:32,680 - root - INFO - Loaded test data



Map:   0%|                                    | 0/478033 [00:00<?, ? examples/s]

...


Map: 100%|████████████████████▉| 478000/478033 [05:19<00:00, 1648.19 examples/s]


Map: 100%|█████████████████████| 478033/478033 [05:19<00:00, 1495.84 examples/s]


2024-04-05 18:51:01,211 - root - INFO - Tokenized test data


2024-04-05 20:12:32,622 - root - INFO - rating
4    183807
5    135028
3     98613
2     36324
1     12792
0     11469
Name: count, dtype: int64


2024-04-05 20:12:33,845 - root - INFO - Saved predictions to csv file


In [11]:
submission_df = pd.read_csv("/kaggle/working/submission.csv")
test_df = pd.read_csv("/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv")
assert(len(submission_df) == len(test_df))